In [1]:
import numpy as np
import pandas as pd

# Population Data Cleaning

In [2]:
# import data
pop_total = pd.read_csv('data-sources/population_total.csv')

In [3]:
def remove_years(df, start_year=2000):
    # remove all year columns from df that are before start_year
    
    df_copy = df.copy()
    
    # collect column headers to be removed
    cols = []
    for col in df_copy:
        if (col.isdigit() and int(col) < start_year):
            cols.append(col)

    # drop columns
    df_copy = df_copy.drop(columns=cols, axis=1)
    
    return df_copy
    
pop_total_short = remove_years(pop_total, start_year=2000)

In [4]:
# remove columns otherwise not required
pop_cols = ['Indicator Name', '2018', 'Indicator Code', 'Country Code']
pop_total_short = pop_total_short.drop(columns=pop_cols, axis=1)

In [5]:
# transpose dataframe
pop_total_short = pop_total_short.transpose()

# change column headers to country names & drop duplicate column
pop_total_short.columns = list(pop_total_short.loc['Country Name' , : ])
pop_total_short = pop_total_short.drop(['Country Name'])

In [6]:
# remove unions
unions = [
    "Central Europe and the Baltics", "Caribbean small states", "East Asia & Pacific (excluding high income)", 
    "Early-demographic dividend", "East Asia & Pacific", "Europe & Central Asia (excluding high income)", 
    "Europe & Central Asia", "Euro area", "European Union", "Fragile and conflict affected situations",
    "High income", "Heavily indebted poor countries (HIPC)", "IBRD only", "IDA & IBRD total", "IDA total",
    "IDA blend", "IDA only", "Latin America & Caribbean", "Not classified" ,"Latin America & Caribbean (excluding high income)", 
    "Least developed countries: UN classification", "Low income", "Lower middle income", "Low & middle income", 
    "Late-demographic dividend", "Middle East & North Africa", "Middle income", 
    "Middle East & North Africa (excluding high income)", "OECD members", "Other small states", 
    "Pre-demographic dividend", "Pacific island small states", "Post-demographic dividend", 
    "Sub-Saharan Africa (excluding high income)", "Sub-Saharan Africa", "Small states", 
    "East Asia & Pacific (IDA & IBRD countries)", "Europe & Central Asia (IDA & IBRD countries)", 
    "Latin America & the Caribbean (IDA & IBRD countries)", "Middle East & North Africa (IDA & IBRD countries)",
    "South Asia (IDA & IBRD)", "Sub-Saharan Africa (IDA & IBRD countries)", "South Asia", "Upper middle income", 
    "World", "Arab World", "North America"
]
pop_total_short = pop_total_short.drop(columns=unions, axis=1)

# remove countries with less than 10M inhabitants in 2000
small_countries = []
for col in pop_total_short:
    if pop_total_short[col]['2000'] <= 10000000:
        small_countries.append(col)

pop_total_short = pop_total_short.drop(columns=small_countries, axis=1)

# drop columns with more than two missing values
pop_total_short = pop_total_short.dropna(axis='columns', thresh=2)

# order by max population in descending order
pop_total_short = pop_total_short.sort_values(by='2017', axis=1, ascending=False)

In [7]:
# calculate population percentage change
pop_perc = pop_total_short.copy()

for col in pop_total_short:
    base_val = pop_total_short[col].iloc[0]
    pop_perc[col] = pop_perc[col]/base_val - 1
    
pop_perc.head()

,China,India,United States,Indonesia,Brazil,Pakistan,Nigeria,Bangladesh,Russian Federation,Mexico,...,Guatemala,Ecuador,Zimbabwe,Cambodia,Cuba,Belgium,Greece,Czech Republic,Portugal,Hungary
2000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2001,0.00729025,0.0174986,0.00994656,0.0140213,0.0140517,0.0222212,0.0254301,0.0191966,-0.00423253,0.0132462,...,0.0235352,0.0177501,0.0117748,0.0205819,0.00321109,0.00344543,0.00521238,-0.00375015,0.00707723,-0.00229116
2002,0.0140618,0.0349045,0.0193604,0.0282104,0.0277455,0.0442587,0.0516109,0.038147,-0.00880315,0.0259137,...,0.047903,0.0351159,0.0227678,0.039694,0.00603207,0.00795366,0.00890392,-0.00567008,0.0126078,-0.00512811
2003,0.0203977,0.0522073,0.0281594,0.0425677,0.0410443,0.0662713,0.0786299,0.056526,-0.0132902,0.038545,...,0.072934,0.0523419,0.03368,0.0576654,0.0084433,0.0121822,0.0113145,-0.00595462,0.0164164,-0.00797368
2004,0.0264762,0.069403,0.037719,0.0570776,0.0539164,0.0884834,0.106591,0.0739182,-0.0172549,0.0518672,...,0.0983784,0.0697663,0.0454303,0.0749668,0.0104213,0.0165723,0.0138197,-0.00565204,0.0188498,-0.010168


In [8]:
# adjust index column
pop_perc = pop_perc.reset_index()
pop_perc = pop_perc.rename(columns={'index': 'Year'})

In [9]:
# add columns for bottom/top lines with distance representing total population
min_population = min(list(pop_total_short.min()))
max_population = float(max(list(pop_total_short.max())))

# scale linearly between min./max. thickness
max_thickness = 0.10
min_thickness = 0.003

slope = (max_thickness - min_thickness)/(max_population - min_population)
intercept = max_thickness - slope*max_population

def thicknessScale(pop, m=slope, n=intercept):
    return m*pop + n

for col in pop_total_short:
    thickness = thicknessScale(pd.to_numeric(pop_total_short[col]))    
    thickness.index = range(18)
    
    # lower limit
    pop_perc[col + '_low'] = pop_perc[col] - thickness/2
    
    # upper limit
    pop_perc[col + '_up'] = pop_perc[col] + thickness/2
    
# thickness scale for d3 legend creation
legend_dims = [thicknessScale(x) for x in [1000000000, 500000000, 10000000]]

print(legend_dims)

[0.07277341022216513, 0.037541859894346594, 0.003014940573084428]


In [10]:
# store data
pop_perc.to_csv('pop_total.csv', index=False)
pop_perc.head()

,Year,China,India,United States,Indonesia,Brazil,Pakistan,Nigeria,Bangladesh,Russian Federation,...,Belgium_low,Belgium_up,Greece_low,Greece_up,Czech Republic_low,Czech Republic_up,Portugal_low,Portugal_up,Hungary_low,Hungary_up
0,2000,0,0,0,0,0,0,0,0,0,...,-0.00151632,0.00151632,-0.00153586,0.00153586,-0.00151646,0.00151646,-0.00151768,0.00151768,-0.0015149,0.0015149
1,2001,0.00729025,0.0174986,0.00994656,0.0140213,0.0140517,0.0222212,0.0254301,0.0191966,-0.00423253,...,0.00192787,0.004963,0.00367454,0.00675023,-0.00526525,-0.00223505,0.00555698,0.00859748,-0.00380524,-0.000777084
2,2002,0.0140618,0.0349045,0.0193604,0.0282104,0.0277455,0.0442587,0.0516109,0.038147,-0.00880315,...,0.00643447,0.00947286,0.00736467,0.0104432,-0.00718449,-0.00415567,0.0110855,0.0141301,-0.00664117,-0.00361505
3,2003,0.0203977,0.0522073,0.0281594,0.0425677,0.0410443,0.0662713,0.0786299,0.056526,-0.0132902,...,0.0106615,0.0137029,0.0097743,0.0128546,-0.00746892,-0.00444031,0.0148928,0.01794,-0.00948571,-0.00646164
4,2004,0.0264762,0.069403,0.037719,0.0570776,0.0539164,0.0884834,0.106591,0.0739182,-0.0172549,...,0.01505,0.0180946,0.0122786,0.0153608,-0.00716645,-0.00413762,0.0173253,0.0203744,-0.0116792,-0.00865674


In [11]:
pop_perc.mean()

Year                        1.111222e+70
China                       5.063269e-02
India                       1.412239e-01
United States               7.933280e-02
Indonesia                   1.241092e-01
Brazil                      1.032062e-01
Pakistan                    2.010849e-01
Nigeria                     2.585130e-01
Bangladesh                  1.338163e-01
Russian Federation         -1.817862e-02
Mexico                      1.294307e-01
Japan                       5.542419e-03
Ethiopia                    2.754389e-01
Philippines                 1.735445e-01
Egypt, Arab Rep.            1.820945e-01
Vietnam                     9.007332e-02
Germany                    -3.889144e-03
Congo, Dem. Rep.            3.261579e-01
Iran, Islamic Rep.          1.114287e-01
Turkey                      1.291748e-01
Thailand                    5.599990e-02
France                      5.600241e-02
United Kingdom              5.548841e-02
Italy                       3.376151e-02
Tanzania        

# Metadata Cleaning

In [12]:
# import data
metadata = pd.read_csv('data-sources/country_metadata.csv')

In [13]:
# remove columns otherwise not required
cols = ['Country Code', 'SpecialNotes']
metadata = metadata.drop(columns=cols, axis=1)

# rename "TableName" column
metadata.columns.values[-1] = 'Country'

# remove all rows with missing valuesp
metadata = metadata.dropna()

In [14]:
# sort by income group
metadata['IncomeGroup'] = pd.Categorical(metadata['IncomeGroup'], 
                                         ["High income", "Upper middle income", "Lower middle income", 
                                          "Low income"])
metadata = metadata.sort_values(by=['IncomeGroup'])

In [15]:
# store data
metadata.to_csv('metadata.csv', index=False)
metadata.head()

,Region,IncomeGroup,Country
0,Latin America & Caribbean,High income,Aruba
173,Europe & Central Asia,High income,Netherlands
79,Europe & Central Asia,High income,United Kingdom
174,Europe & Central Asia,High income,Norway
76,Europe & Central Asia,High income,Faroe Islands
